## Этап 1. Установка библиотек и импорт данных

In [1]:
# Cкачиваем необходимые библиотеки для работы
!pip install pandas numpy scikit-learn

In [2]:
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.decomposition import PCA
import pandas as pd
import glob
import os

Подключаемся в Google Drive, чтобы работать с файлами из Google Colaboratory


In [3]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


Указываем путь к корневым папкам с данными test и train

__Обратите внимание:__ если вы создали папку с названием отличным от _"Хакатон Альфа-банка. Кейс 2"_, то необходимо поменять путь к директории и ссылки ниже

In [45]:
# Путь к папке с тренировочными данными
path_train = '/content/drive/MyDrive/Alfa Hack. Final. Data/private/margin_propensity/train'
# Путь к папке с тестовыми данными
path_test = '/content/drive/MyDrive/Alfa Hack. Final. Data/private/margin_propensity/test'

Получим список файлов в каждой из папок с данными

In [46]:
print(*[f'Список файлов в репозитории {x.split("/")[-1]}: {sorted(os.listdir(x), key=lambda x: len(x))}' for x in [path_train, path_test]], sep='\n')

Список файлов в репозитории train: ['train_1.csv', 'train_2.csv', 'train_3.csv', 'train_4.csv', 'train_5.csv', 'train_6.csv', 'train_7.csv', 'train_8.csv', 'train_9.csv', 'train_10.csv']
Список файлов в репозитории test: ['test_1.csv', 'test_2.csv', 'test_3.csv', 'test_4.csv', 'test_5.csv', 'test_6.csv', 'test_7.csv', 'test_8.csv', 'test_9.csv', 'test_10.csv']


Объединим тренировочные данные в единый датасет train

In [47]:
# Получим список путей к файлам в папке train
filenames_train = glob.glob(path_train + "/*.csv")

# Инициализируем PCA для сжатия данных
pca = PCA(n_components=50, random_state=42)
# Считываем первый датасет для обучения
learn = pd.read_csv(filenames_train[0])
# Обучаем PCA на первом датасете без искомых признаков
pca.fit(learn.drop(['target', 'smpl', 'id'], axis=1))
# Создаем список с названием новых фичей
new_columns = [f'feature_{i}' for i in range(1, 51)]


def compression(filename, features=['target', 'smpl', 'id']):
  # Переводим переменные в глобальную область видимости
  global data, base_info, transformed_data, result
  # Считываем файл данных
  data = pd.read_csv(filename)
  # Отделяем базовые данные
  base_info = data[features]
  # Трансформируем данные
  transformed_data = pd.DataFrame(pca.transform(data.drop(features, axis=1)))
  # Объединяем изначальную часть и преобразованные данные
  result = pd.concat([base_info, transformed_data], ignore_index=True, axis=1)
  # Обновляем названия признаков
  result.columns = [*features, *new_columns]
  # Возвращаем результат обработки
  return result


# Создадим список для записи считанных и обработанных файлов train
data_files_train = []


# Считаем и обработаем все файлы train, после чего добавим их в список
for filename in filenames_train:
  # Добавляем в список очищенный датасет
  data_files_train.append(compression(filename))

In [48]:
# Объединим тренировочные данные в единый датасет
train_data = pd.concat(data_files_train, ignore_index=True)

# Удалим ненужные переменные для очистки памяти
del learn, data_files_train, filenames_train, path_train, data, base_info, transformed_data, result

# Выведем информацию о размерности полученных тренировочных данных
display('Размерность полных тренировочных данных составляет: {} строк и {} столбца'.format(*train_data.shape))

# Выведем первые 5 строк тренировочных данных
display(train_data.head())

# Удостоверимся, что перед нами данные только из выборки train
display(train_data['smpl'].value_counts())

'Размерность полных тренировочных данных составляет: 657507 строк и 53 столбца'

,target,smpl,id,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,...,feature_41,feature_42,feature_43,feature_44,feature_45,feature_46,feature_47,feature_48,feature_49,feature_50
0,0,train,0,15.653318,20.211694,20.741967,7.660992,-5.700876,3.863138,0.573271,...,0.568459,-0.140383,1.249398,-0.507152,0.614405,0.582061,0.050516,-0.034203,0.798179,-0.301194
1,0,train,1,16.120384,21.313523,20.427459,6.881421,-5.116853,3.147272,1.052858,...,1.838063,-0.717137,0.015423,0.597851,0.806664,-2.668230,-0.326466,0.584311,1.544434,0.955334
2,0,train,2,30.132003,-41.906895,4.079529,-19.850274,-4.956666,-6.477284,5.505437,...,-0.987814,-0.117918,0.715920,-0.850103,1.688766,0.529400,0.129510,-0.971389,-1.586823,-0.935411
3,1,train,3,-26.224942,-9.829306,-14.934313,3.468490,15.558536,-13.550163,4.248066,...,0.601755,-0.673910,-0.309764,-0.422741,0.138324,0.993551,0.351355,-0.952787,-2.296593,-0.940669
4,0,train,4,-27.743187,-11.085753,10.881654,-19.460133,-0.946568,-8.294459,4.659880,...,-2.486344,0.303427,-0.495271,1.159214,-0.652791,-0.540171,-3.277987,1.453622,-0.221676,-0.112782


,count
smpl,
train,657507


Объединим тестовые данные в единый датасет test

In [49]:
# Получим список путей к файлам в папке test
filenames_test = glob.glob(path_test + "/*.csv")

# Создадим список для записи считанных файлов test
data_files_test = []

# Считаем и обработаем все файлы test, после чего добавим их в список
for filename in filenames_test:
  # Добавляем в список очищенный датасет
  data_files_test.append(compression(filename, features=['smpl', 'id']))

In [50]:
# Объединим тестовые данные в единый датасет
test_data = pd.concat(data_files_test, ignore_index=True)

# Удалим ненужные переменные для очистки памяти
del data_files_test, filenames_test, path_test, data, base_info, transformed_data, result

# Выведем информацию о размерности полученных тестовых данных
display('Размерность полных тестовых данных составляет: {} строка и {} столбец'.format(*test_data.shape))

# Выведем первые 5 строк тренировочных данных
display(test_data.head())

# Удостоверимся, что перед нами данные только из выборки test
display(test_data['smpl'].value_counts())

'Размерность полных тестовых данных составляет: 342493 строка и 52 столбец'

,smpl,id,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,...,feature_41,feature_42,feature_43,feature_44,feature_45,feature_46,feature_47,feature_48,feature_49,feature_50
0,test,657507,48.746695,-36.252949,8.473212,-9.117410,18.304149,25.502374,1.593336,-1.641986,...,-0.515543,0.720977,1.543290,-0.389463,-0.335501,-2.664579,-1.048653,-0.012752,-0.536204,1.250421
1,test,657508,-25.200177,-9.261709,17.283789,-1.301911,7.019383,-6.140478,-1.344413,0.968174,...,-0.402337,0.742431,-0.744452,-0.837695,-2.456574,1.338317,-0.600280,-1.935004,-0.756764,1.528634
2,test,657509,61.030411,0.646227,6.030876,-19.611813,-9.885349,-5.774910,6.914830,-8.187758,...,0.641588,0.707648,-0.672994,-0.489509,-1.176057,0.367463,-2.089056,0.182827,-0.455541,0.043194
3,test,657510,65.228284,0.707193,7.294471,-22.958943,-7.776763,-8.708117,2.163809,-2.858376,...,-1.782085,1.228686,0.020635,-3.283654,-1.028727,-0.387088,-3.109345,-2.041788,2.013974,1.893102
4,test,657511,-10.351484,29.059815,-14.628473,-4.982259,1.158835,-9.141879,8.995381,1.210989,...,-2.193888,1.443446,-0.353651,-0.830208,1.256887,-0.799003,0.285104,0.243507,-1.247114,-1.452891


,count
smpl,
test,342493


## Этап 2. Работа с моделью

In [51]:
# Выделим признаки для обучения и искомый признак в датасете train
X = train_data.drop(['target', 'smpl'], axis=1)
y = train_data['target']

# Разбиваем тренировочные данные на валидационный и тренировочный датасет в соотношении 7 к 3
# Cтратифицируем выборки по искомому признаку, фиксируем random_state для воспроизводимости
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y, test_size=0.3)

# Обучаем модель на тренировочной части тренировочных данных
gbdt_clf = HistGradientBoostingClassifier(min_samples_leaf=1,
                                          max_depth=20,
                                          max_iter=125,
                                          random_state=42).fit(X_train, y_train)

# Получаем предсказание с вероятностями для валидационной части тренировочного датасета
y_pred = gbdt_clf.predict_proba(X_test)

# Переводим предсказание в формат Series
y_pred = pd.Series(y_pred[:, 1])

# Высчитываем метрику roc-auc по валидационным данным
print(f'Метрика roc-auc на валидационных данных имеет значение: {roc_auc_score(y_test, y_pred)}')

Метрика roc-auc на валидационных данных имеет значение: 0.8142327058753533


In [52]:
# Удалим ненужные переменные для очистки памяти
del X_train, X_test, y_train, y_test, gbdt_clf, y_pred

## Этап 3. Предсказание для тестовых данных


In [53]:
# Обучим модель на полных тренировочных данных
gbdt_clf = HistGradientBoostingClassifier(min_samples_leaf=1,
                                          max_depth=20,
                                          max_iter=125,
                                          random_state=42).fit(X, y)

# Для предсказания используем тестовый датасет с исключенным признаком smpl
y_test_pred = gbdt_clf.predict_proba(test_data.drop('smpl', axis=1))

# Переведем предсказание в формат Series
y_test_pred = pd.Series(y_test_pred[:, 1])

# Добавим данные предсказания к датасету
test_data['target'] = y_test_pred

# Сохраняем итоговые данные об id и предсказаниях в формате csv
test_data[['id', 'target']].to_csv('test_submission_margin_propensity.csv', index=False)

Baseline создан компанией Changellenge >> - ведущей организацией по кейсам в России, для хакатона Alfa Hack.  

Запрещено распространение данного файла, файлов данных в форматах csv, а также текста кейса без письменного разрешения владельца авторских прав в лице компании Changellenge>>.